# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Using-the-AfSIS-DB-API" data-toc-modified-id="Using-the-AfSIS-DB-API-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Using the AfSIS DB API</a></div><div class="lev1 toc-item"><a href="#Introduction" data-toc-modified-id="Introduction-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Introduction</a></div><div class="lev2 toc-item"><a href="#Background" data-toc-modified-id="Background-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Background</a></div><div class="lev2 toc-item"><a href="#Purpose-of-this-Notebook" data-toc-modified-id="Purpose-of-this-Notebook-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Purpose of this Notebook</a></div><div class="lev1 toc-item"><a href="#Load-Libraries-and-Credentials" data-toc-modified-id="Load-Libraries-and-Credentials-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Libraries and Credentials</a></div><div class="lev1 toc-item"><a href="#Downloading-by-API" data-toc-modified-id="Downloading-by-API-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Downloading by API</a></div><div class="lev2 toc-item"><a href="#Documentation" data-toc-modified-id="Documentation-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Documentation</a></div><div class="lev2 toc-item"><a href="#Making-API-Calls" data-toc-modified-id="Making-API-Calls-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Making API Calls</a></div><div class="lev3 toc-item"><a href="#Basic-Call" data-toc-modified-id="Basic-Call-421"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Basic Call</a></div><div class="lev3 toc-item"><a href="#Handling-Pagination" data-toc-modified-id="Handling-Pagination-422"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Handling Pagination</a></div><div class="lev2 toc-item"><a href="#Filtering" data-toc-modified-id="Filtering-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Filtering</a></div><div class="lev3 toc-item"><a href="#Machine-Type" data-toc-modified-id="Machine-Type-431"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Machine Type</a></div><div class="lev3 toc-item"><a href="#Group" data-toc-modified-id="Group-432"><span class="toc-item-num">4.3.2&nbsp;&nbsp;</span>Group</a></div><div class="lev2 toc-item"><a href="#Downloading" data-toc-modified-id="Downloading-44"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Downloading</a></div><div class="lev2 toc-item"><a href="#Other-tables" data-toc-modified-id="Other-tables-45"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Other tables</a></div>

# Using the AfSIS DB API

QED | https://qed.ai

# Introduction

## Background 

[AfSIS DB](https://afsisdb.qed.ai) is a data management system for soil laboratory data, developed by QED for the Africa Soil Information Service (AfSIS). The key features that it provides include:

* Database storage for both relational and binary blob data, using PostgreSQL and S3, respectively.
* Web interface for uploading data, with support for server-side input validation.
* Web interface for searching and exporting data, including API support.
* Optional linking of laboratory data with field metadata collected using [Mobilesurvey](https://mobilesurvey.qed.ai).
* Access controls for groups and public vs. private data.

Because AfSIS DB runs on the web, it is cross-platform compatible. At the time of this writing, AfSIS DB now stores over 175,000 records of soil chemistry measurements unique to Africa, mostly from the AfSIS Phase II focus contries of Tanzania, Ghana, Nigeria, and Ethiopia. 

To register for an account on AfSIS DB, please submit a request for an invite at https://afsisdb.qed.ai.

You are also invited to view our tutorial videos on YouTube:

[![Introduction to AfSIS DB](https://qed.ai/afsisdb/QED_Intro_AfSISDB.jpg)](https://www.youtube.com/watch?v=CswaipfQxCs "Introduction to AfSIS DB")

[![Using the AfSIS DB API](https://qed.ai/afsisdb/QED_Using_AfSISDB_API.jpg)](https://www.youtube.com/watch?v=z46n1yaSXhU "Using the AfSIS DB API")


## Purpose of this Notebook

In this notebook, we demonstrate the usage of API calls to fetch data from AfSIS DB programmatically, using R. Another notebook demonstrating analogous operations in Python will be added to this repository in the near future.

# Load Libraries and Credentials

First we must load your AfSIS DB credentials. Here I have stored them in a config file named "config.ini", the contents of which are simply:

    username = "your_username"
    password = "your_password"
   
where you should replace the strings here with your own credentials. Do include the quotation marks. Make sure this file is stored in the same working directory as this notebook, and then execute the sourcing line below.

In [ ]:
source("config.ini")

Next we install and load some packages. 
* "httr" is used for working with HTTP verbs, and installs "jsonlite" as a dependency. 
* "lubridate" is used for date-time manipulation and arithemtic.
* "plyr" is a popular general toolkit for dataset manipulation.

In [ ]:
install.packages(c("httr", "jsonlite", "lubridate", "plyr"))

In [ ]:
library(httr)
library(jsonlite)
library(plyr)

# Downloading by API

## Documentation

To view the full API documentation, please register for an account on AfSIS DB, log in, and *then* visit the URL: https://afsisdb.qed.ai/cabinet/api-docs/. You will see auto-generated and interactive documentation for every table, such as the following:

![AfSISDB_API](https://qed.ai/AfSISDB_API_mini.jpg "AfSIS DB API")

## Making API Calls

### Basic Call

In the following function, we make an API call to AfSIS DB and merge the matching query results into a data frame, replacing null values with R's NA symbol.

In [ ]:
# row bind data with missing cells
rbind_apicall <- function(apicall) {
    apicall_df_singlepage <- rbind.fill(lapply(content(apicall)$results, 
        function(f) { as.data.frame(Filter(Negate(is.null), f)) } ))
    return(apicall_df_singlepage)
}

### Handling Pagination

Queries to AfSIS DB are currently paginated such that one has finer control over the response size. To iterate through all the pages and merge them into one dataframe, we can use a for loop, as shown below. We add a simple progress bar to visualize the progress.

This is the "api_to_df" function that will be used in the remainder of this tutorial.

In [ ]:
# returns dataframe from an API call, stitching together results from all pages
api_to_df <- function(url, username, password){
    apicall_data <- GET(url, authenticate(username,password))
    apicall_data_df <- rbind_apicall(apicall_data)
    apicall_data_content <- content(apicall_data)
    num_pages <- ceiling(apicall_data_content$count/10)
    progress_bar = txtProgressBar(min = 1, max = num_pages-1, initial = 0, style=3) 

    for (i in 2:num_pages) {
        setTxtProgressBar(progress_bar,i)
        apicall_data_next <- GET(paste(url, paste("page=", i,sep=""),sep="&"), 
                                 authenticate(username,password))
        apicall_data_df_next <- rbind_apicall(apicall_data_next)
        apicall_data_df <- rbind.fill(apicall_data_df, apicall_data_df_next)                            
    }
    return(apicall_data_df)
}

## Filtering

### Machine Type

The machines that AfSIS DB currently supports are the Bruker alpha_znse, alpha_kbr, mpa, and htsxt. 

In this example, we can search only for spectral files generated by the MPA, an FT-NIR spectrometer.

In [ ]:
mpa_url <- "http://afsisdb.qed.ai/cabinet/api/sample/?machine=mpa"
mpa_df <- api_to_df(mpa_url, username, password)

We can inspect summary statistics about the data frame.

In [ ]:
str(mpa_df)

### Group

Here we demonstrate filtering by Group affiliation.

In [ ]:
TanSIS_mpa_url <- "http://afsisdb.qed.ai/cabinet/api/sample/?group=TanSIS&machine=mpa"
TanSIS_mpa_df <- api_to_df(TanSIS_mpa_url, username, password)

In [ ]:
str(TanSIS_mpa_df)

## Downloading

* Auto-generated URLs for downloading spectral files are inside the data frame. 
* To download the spectral files, you must explicitly invoke the download operation on each URL.
* As a demonstration, here we will only download the first 10 OPUS files from the "mpa_df" data frame.
* There is some logic to force filenames to be formatted as "ssn.subsample_id". This is due to inconsistency in the naming schemes used by different labs.

In [ ]:
progress_bar = txtProgressBar(min = 1, max = 10, initial = 0, style=3) 
setTxtProgressBar(progress_bar,0)
for (i in 1:10) {
    file_name = paste(as.character(mpa_df$ssn[i]), 
                      as.character(mpa_df$subsample_id[i]), sep=".")
    download.file(as.character(mpa_df$binary_file[i]), method="curl", destfile=file_name) 
    setTxtProgressBar(progress_bar,i)
}

## Other tables

Besides spectroscopy data, AfSIS DB also houses meaurements of wet chemistry, LDPSA, Carbon and Nitrogen, soil moisture, georeferences, and more. All tables can be queried in a similar fashion, and the resulting data frames can be joined by their Soil Sample Number (SSN) as long as all labs use the same SSN when interacting with AfSIS DB. below are some examples of fetching wet chemistry data provided by One Acre Fund.

In [ ]:
wetchem_1af_url <- "https://afsisdb.qed.ai/cabinet/api/wetchemistry/?group=1AF"

In [ ]:
wetchem_1af_data <- api_to_df(wetchem_1af_url, username, password)

In [ ]:
str(wetchem_1af_data)